In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-anj_v5h1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-anj_v5h1
  Resolved https://github.com/huggingface/transformers to commit 8b73799500a69242d0afa73c1256bcaf7a3fff37
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11430792 sha256=44dcf910e27ab15dbd8f1853e0cdaa632b7f87ff6dd084402a350e1f44692858
  Stored in directory: /tmp/pip-ephem-wheel-cache-8lanf6vy/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers


In [ ]:
!pip install accelerate --quiet
!pip install qwen-vl-utils --quiet
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 25.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
import shutil
from tqdm import tqdm

zip_path = '/content/drive/MyDrive/OmniMedVQA/Images.zip'
extract_dir = '/content/Images'

os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zipf:
    file_list = zipf.namelist()
    for member in tqdm(file_list, desc="Unzipping images"):
        # Skip directories
        if member.endswith('/'):
            continue
        # Remove leading "Images/" from the path
        if member.startswith('Images/'):
            target = member[len('Images/'):]
        else:
            target = member
        target_path = os.path.join(extract_dir, target)
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        with zipf.open(member) as source, open(target_path, "wb") as dest:
            shutil.copyfileobj(source, dest)

Unzipping images: 100%|██████████| 6675/6675 [00:10<00:00, 665.44it/s] 


In [ ]:
import pandas as pd
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, GenerationConfig
from qwen_vl_utils import process_vision_info
import torch
from tqdm import tqdm
import os

# Check CSV directory and files
csv_dir = '/content/drive/MyDrive/OmniMedVQA/CSV_Files'
if not os.path.exists(csv_dir):
    raise FileNotFoundError(f"CSV directory does not exist: {csv_dir}")
print("CSV directory exists.")
print("Files in CSV directory:", os.listdir(csv_dir))

# Specify paths and desired sizes
csv_paths = {
    'train': f"{csv_dir}/train.csv",
    'val': f"{csv_dir}/val.csv",
    'test_mri': f"{csv_dir}/test_mri.csv",
    'test_ct': f"{csv_dir}/test_ct.csv",
    'test_xray': f"{csv_dir}/test_xray.csv"
}
train_size = 1000
val_size = 200
test_size = 200

# Sampling function
def sample_and_save(csv_in, csv_out, n):
    df = pd.read_csv(csv_in)
    if len(df) > n:
        df = df.sample(n=n, random_state=42).reset_index(drop=True)
    df.to_csv(csv_out, index=False)
    print(f"Saved {csv_out} with {len(df)} rows.")

# Sample and save splits
sample_and_save(csv_paths['train'], f"{csv_dir}/train_sampled.csv", train_size)
sample_and_save(csv_paths['val'], f"{csv_dir}/val_sampled.csv", val_size)
sample_and_save(csv_paths['test_mri'], f"{csv_dir}/test_mri_sampled.csv", test_size)
sample_and_save(csv_paths['test_ct'], f"{csv_dir}/test_ct_sampled.csv", test_size)
sample_and_save(csv_paths['test_xray'], f"{csv_dir}/test_xray_sampled.csv", test_size)

# Update paths to sampled files
sampled_csv_paths = {
    'train': f"{csv_dir}/train_sampled.csv",
    'val': f"{csv_dir}/val_sampled.csv",
    'test_mri': f"{csv_dir}/test_mri_sampled.csv",
    'test_ct': f"{csv_dir}/test_ct_sampled.csv",
    'test_xray': f"{csv_dir}/test_xray_sampled.csv"
}

# Load MedVLM-R1 model and processor
MODEL_PATH = 'JZPeterPan/MedVLM-R1'
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)
processor = AutoProcessor.from_pretrained(MODEL_PATH)

temp_generation_config = GenerationConfig(
    max_new_tokens=128,  # Limit reasoning to 128 tokens for efficiency
    do_sample=False,
    temperature=1,
    num_return_sequences=1,
    pad_token_id=151643,
)

def construct_prompt(row):
    return (
        f"{row['question']}\n"
        f"A) {row['option_A']}\n"
        f"B) {row['option_B']}\n"
        f"C) {row['option_C']}\n"
        f"D) {row['option_D']}\n"
        "First, think through the question step by step, enclose your reasoning process in <think>...</think> tags. "
        "Then provide the correct single-letter choice (A, B, C, D,...) inside <answer>...</answer> tags."
    )

def generate_cot_for_sample(image_path, question_text):
    try:
        message = [{
            'role': 'user',
            'content': [
                {'type': 'image', 'image': f'file://{image_path}'},
                {'type': 'text', 'text': question_text}
            ]
        }]
        text = processor.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info(message)
        if not image_inputs or image_inputs[0] is None:
            return "ERROR: Empty or invalid image_inputs"
        inputs = processor(
            text=text,
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors='pt',
        ).to(model.device)
        if 'input_ids' not in inputs or inputs.input_ids.shape[0] == 0:
            return "ERROR: Missing or empty input_ids"
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                use_cache=True,
                generation_config=temp_generation_config
            )
        if generated_ids is None or generated_ids.shape[0] == 0:
            return "ERROR: Empty generated_ids"
        generated_ids_trimmed = [
            out_ids[len(in_ids):]
            for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        return output_text[0]
    except Exception as e:
        return f"ERROR: {type(e).__name__}: {str(e)}"

def add_cot_to_df(df, image_col, cot_col):
    df['abs_image_path'] = df[image_col].apply(lambda x: '/content/' + x if not x.startswith('/content/') else x)
    df = df[df['abs_image_path'].apply(os.path.isfile)].reset_index(drop=True)
    df[cot_col] = ''
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating CoT"):
        image_path = row['abs_image_path']
        prompt = construct_prompt(row)
        cot_output = generate_cot_for_sample(image_path, prompt)
        df.at[idx, cot_col] = cot_output
        if (idx + 1) % 5 == 0 or idx == len(df) - 1:
            print(f'Processed {idx+1}/{len(df)}')
    return df

# Process each sampled split
for split, csv_path in sampled_csv_paths.items():
    print(f"\n Processing {split} split...")
    df = pd.read_csv(csv_path)
    df = add_cot_to_df(df, image_col='image_path', cot_col='medvlm_r1_cot')
    out_path = csv_path.replace('.csv', '_with_cot.csv')
    df.to_csv(out_path, index=False)
    print(f"Saved {out_path} with CoT outputs.")

print("All splits processed and saved with MedVLM-R1 CoT outputs.")

CSV directory exists.
Files in CSV directory: ['test_ct.csv', 'test_xray.csv', 'train.csv', 'test_mri.csv', 'val.csv']
Saved /content/drive/MyDrive/OmniMedVQA/CSV_Files/train_sampled.csv with 1000 rows.
Saved /content/drive/MyDrive/OmniMedVQA/CSV_Files/val_sampled.csv with 200 rows.
Saved /content/drive/MyDrive/OmniMedVQA/CSV_Files/test_mri_sampled.csv with 200 rows.
Saved /content/drive/MyDrive/OmniMedVQA/CSV_Files/test_ct_sampled.csv with 200 rows.
Saved /content/drive/MyDrive/OmniMedVQA/CSV_Files/test_xray_sampled.csv with 200 rows.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

Processed 190/200


Generating CoT:  98%|█████████▊| 195/200 [15:27<00:21,  4.29s/it]

Processed 195/200


Generating CoT: 100%|██████████| 200/200 [15:53<00:00,  4.77s/it]

Processed 200/200
Saved /content/drive/MyDrive/OmniMedVQA/CSV_Files/test_xray_sampled_with_cot.csv with CoT outputs.
All splits processed and saved with MedVLM-R1 CoT outputs.
